# Un mal estudio...

In [1]:
import pandas as pd
import numpy as np

In [4]:
# Fuente: 
# https://datos.gob.es/es/catalogo/80a2358d-d745-4632-a422-3556d90e0b47
# Publicador  Junta de Castilla y León Público
# Nivel de administración  Administración Autonómica
# Licencia https://creativecommons.org/licenses/by/4.0/deed.es_ES

df = pd.read_csv("ResultadosElectorales2022.csv",encoding="cp1250",sep=";")
##
print(df.columns)
print(df.shape)
print(df.head(10))
##
print(len(np.unique(df['Código de mesa'])))
##
print(np.unique(df.Partido))


Index(['Código de mesa', 'Provincia', 'Código \nmunicipio', 'Municipio',
       'Distrito', 'Sección', 'Mesa', 'Censo', 'Primer Avance',
       'Segundo Avance', 'Total Votantes', 'Votos Nulos', 'Votos Blanco',
       'Partido', 'Nş Votos'],
      dtype='object')
(51784, 15)
   Código de mesa Provincia  Código \nmunicipio Municipio  Distrito  Sección  \
0  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
1  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
2  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
3  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
4  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
5  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
6  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
7  05-001-1-001-U     AVILA                   1   ADANERO         1        1   
8  0

In [26]:
# A1. Si en cada mesa hay 3 voluntarios, ¿cuántos voluntarios han sido necesarios?
print((len(df)+1)*3)#Faltaba sumar el 0 de la primera fila


155355


In [36]:
# A2. ¿Cuál es la participación media de votantes?
(df["Total Votantes"]/df["Censo"]).mean()


0.654325195724643

In [61]:
# A3. ¿En qué porcentaje de municipios se ha presentado el partido "ESPAÑA VACIADA"?
dff = df[(df["Partido"].isin(["ESPAŃA VACIADA"]))][["Municipio"]]
municipios = np.unique(dff.values)
print(len(municipios)/len(df))


0.02218832071682373


,Código de mesa,Provincia,Código \nmunicipio,Municipio,Distrito,Sección,Mesa,Censo,Primer Avance,Segundo Avance,Total Votantes,Votos Nulos,Votos Blanco,Partido,Nş Votos
0,05-001-1-001-U,AVILA,1,ADANERO,1,1,U,173,49,85,108,2,2,PP,41
1,05-001-1-001-U,AVILA,1,ADANERO,1,1,U,173,49,85,108,2,2,VOX,23
2,05-001-1-001-U,AVILA,1,ADANERO,1,1,U,173,49,85,108,2,2,P.S.O.E.,23
3,05-001-1-001-U,AVILA,1,ADANERO,1,1,U,173,49,85,108,2,2,XAV,13
4,05-001-1-001-U,AVILA,1,ADANERO,1,1,U,173,49,85,108,2,2,Cs,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51779,49-275-2-017-U,ZAMORA,275,ZAMORA,2,17,U,659,211,296,365,2,2,Cs,9
51780,49-275-2-017-U,ZAMORA,275,ZAMORA,2,17,U,659,211,296,365,2,2,PACMA,3
51781,49-275-2-017-U,ZAMORA,275,ZAMORA,2,17,U,659,211,296,365,2,2,POR ZAMORA,2
51782,49-275-2-017-U,ZAMORA,275,ZAMORA,2,17,U,659,211,296,365,2,2,PREPAL,1


In [46]:
# A4. ¿Cuál es el censo por provincia?
df2 = df.groupby([df.columns[1],df.columns[2]]).first().reset_index()
print(df2.groupby("Provincia").agg({"Censo":"sum"}))#En vez de hacer un max se deberia hacer sum del Censo

            Censo
Provincia        
AVILA       51743
BURGOS      68532
LEÓN        86059
PALENCIA    41139
SALAMANCA   83999
SEGOVIA     47464
SORIA       24694
VALLADOLID  59610
ZAMORA      67896


In [56]:
# A5. ¿En cuántas mesas electorales el "PP" ha obtenido más votos que el "P.S.O.E." o "PSOE"?
dff = df[(df["Partido"].isin(["PP","P.S.O.E.","PSOE"]))][["Municipio","Código \nmunicipio","Partido",df.columns[-1]]]
dfpartidos = pd.pivot_table(dff,index="Partido",columns=["Municipio","Código \nmunicipio"])
respectoPSOE = (dfpartidos.loc["PP"]>dfpartidos.loc["P.S.O.E."]).sum()
respectoPSOE += (dfpartidos.loc["PP"]>dfpartidos.loc["PSOE"]).sum()

print("El PP ha obtenido más votos que el PSOE en %i mesas electorales sobre un total de %i mesas. "%(respectoPSOE,dff.shape[0]//2))


El PP ha obtenido más votos que el PSOE en 1661 mesas electorales sobre un total de 4531 mesas. 


In [62]:
# A6. ¿Ordena las comunidades según su número de mesas electorales?
comunidades, mesas = np.unique(df.groupby(["Provincia",df.index]).first().index.get_level_values(0),return_counts=True)#Necesitamos mirar la cantidad del indice
mesas = sorted(mesas,reverse=True)
comunmesas = dict(zip(comunidades,mesas))
print("Comunidades y número de mesas:\n", comunmesas)



Comunidades y número de mesas:
 {'AVILA': 9997, 'BURGOS': 9576, 'LEÓN': 7975, 'PALENCIA': 7290, 'SALAMANCA': 4225, 'SEGOVIA': 4059, 'SORIA': 3560, 'VALLADOLID': 2552, 'ZAMORA': 2550}


(array(['AVILA', 'BURGOS', 'LEÓN', 'PALENCIA', 'SALAMANCA', 'SEGOVIA',
        'SORIA', 'VALLADOLID', 'ZAMORA'], dtype=object),
 [9997, 9576, 7975, 7290, 4225, 4059, 3560, 2552, 2550])